In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count.csv')

In [ ]:
# Pickle 파일 읽기
with open('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_embeddings_full.pkl', 'rb') as file:
    data = pickle.load(file)

# DataFrame으로 변환
df_embedding = pd.DataFrame(data, columns=['embedding', 'track_id'])

# track_id를 정수로 변환 (필요하다면)
df_embedding['track_id'] = df_embedding['track_id'].astype(int)

# 'embedding' 열을 768차원의 각 차원으로 나누기
# df_embedding[['embedding_{}'.format(i) for i in range(768)]] = pd.DataFrame(df_embedding['embedding'].tolist(), index=df_embedding.index)

# 'embedding' 열 삭제
# df_embedding = df_embedding.drop(['embedding'], axis=1)

# DataFrame 확인
print(df_embedding.head())


                                           embedding  track_id
0  [0.012072664, 0.17292306, 0.0061238254, 0.0707...         2
1  [-0.17554894, 0.24209566, 0.4195969, -0.185033...         8
2  [-0.096951924, 0.0034472912, 0.005701333, 0.01...      1524
3  [-0.21775067, 0.244962, 0.24090661, 0.1647732,...      1785
4  [-0.069424234, -0.016805744, 0.21406727, -0.27...      1787


In [ ]:
print(df.shape)
df = pd.merge(df, df_embedding, on='track_id', how='inner')
df.shape

(4654011, 4)


(4653095, 5)

In [ ]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])
df_embedding['track_id'] = lyrics_encoder.fit_transform(df_embedding['track_id'])

In [ ]:
lyrics_dict = dict(zip(df_embedding['track_id'], df_embedding['embedding']))

In [ ]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
df.loc[df['listen_count_bin'] == '10~2704', 'listen_count_bin'] = 10
df['listen_count_bin'] = df['listen_count_bin'].astype(int)
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)
    lyrics_embeddings = torch.tensor(np.vstack(dataset['embedding'].values), dtype=torch.float)
    return users, items, ratings, lyrics_embeddings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings, train_lyrics_embeddings = df_to_tensor(train_df)
test_users, test_items, test_ratings,test_lyrics_embeddings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings,train_lyrics_embeddings)
test_data = TensorDataset(test_users, test_items, test_ratings,test_lyrics_embeddings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

24458
31000


In [ ]:
# NCF 모델 정의
class NCF_embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF_embedding, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.lyrics_embedding = nn.Linear(768, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 3, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item, lyrics_embedding):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        lyrics_embedding = self.lyrics_embedding(lyrics_embedding.reshape(lyrics_embedding.shape[0],768))
        x = torch.cat((user_embedding, item_embedding,lyrics_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_embedding(embedding,n_epoch):
  # NCF 모델 정의 및 GPU로 이동
  model = NCF_embedding(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for user, item, rating,lyrics_embedding in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating,lyrics_embedding = user.to(device), item.to(device), rating.to(device),lyrics_embedding.to(device)  # GPU로 이동
          output = model(user, item,lyrics_embedding.float())
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
      model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _,lyrics_embedding in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item,lyrics_embedding = user.to(device), item.to(device),lyrics_embedding.to(device)  # GPU로 이동
              output = model(user, item,lyrics_embedding)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')


Using device: cuda


In [ ]:
train_embedding(256,30)

Epoch 1/30: 100%|██████████| 14549/14549 [01:49<00:00, 132.81it/s]


Epoch 1/30, Avg. Loss: 5.9565


Testing Epoch 1: 100%|██████████| 3638/3638 [00:17<00:00, 203.57it/s]


Epoch 1/30, RMSE on test set: 2.3912988925296563


Epoch 2/30: 100%|██████████| 14549/14549 [01:53<00:00, 127.74it/s]


...

Epoch 25/30, Avg. Loss: 3.3517


Testing Epoch 25: 100%|██████████| 3638/3638 [00:18<00:00, 201.88it/s]


Epoch 25/30, RMSE on test set: 2.394815272480803


Epoch 26/30:   3%|▎         | 374/14549 [00:03<02:04, 114.12it/s]


KeyboardInterrupt: ignored

In [ ]:
train_embedding(512,15)

Epoch 1/15: 100%|██████████| 14541/14541 [02:01<00:00, 119.40it/s]


Epoch 1/15, Avg. Loss: 5.9390


Testing Epoch 1: 100%|██████████| 3636/3636 [00:17<00:00, 203.70it/s]


Epoch 1/15, RMSE on test set: 2.3858880138395118


Epoch 2/15: 100%|██████████| 14541/14541 [02:07<00:00, 114.36it/s]


...

Epoch 14/15, RMSE on test set: 2.2951758469849874


Epoch 15/15: 100%|██████████| 14541/14541 [02:07<00:00, 114.28it/s]


Epoch 15/15, Avg. Loss: 3.5913


Testing Epoch 15: 100%|██████████| 3636/3636 [00:17<00:00, 206.88it/s]

Epoch 15/15, RMSE on test set: 2.3232439470455324


In [ ]:
train_embedding(768,15)

Epoch 1/15: 100%|██████████| 14541/14541 [02:16<00:00, 106.47it/s]


Epoch 1/15, Avg. Loss: 5.9386


Testing Epoch 1: 100%|██████████| 3636/3636 [00:17<00:00, 206.21it/s]


Epoch 1/15, RMSE on test set: 2.389300544746402


Epoch 2/15: 100%|██████████| 14541/14541 [02:22<00:00, 102.40it/s]


...

Epoch 14/15, RMSE on test set: 2.2848379384917736


Epoch 15/15: 100%|██████████| 14541/14541 [02:22<00:00, 102.04it/s]


Epoch 15/15, Avg. Loss: 3.3220


Testing Epoch 15: 100%|██████████| 3636/3636 [00:17<00:00, 206.49it/s]

Epoch 15/15, RMSE on test set: 2.298338231083951
